## Манифест

#### 1)Заголовки

Как правило, в начале документа идет глобальный заголовок - будем считать его заголовком 0 уровня.

Затем могут идти подзаголовки, начинающиеся со слов Глава, Раздел, Секция, Параграф, Статья + нумерация. Это заголовки n-го уровня, в зависимости от нумерации. (Раздел 1 - 1 уровень, Раздел/Подраздел 1.1 - 2 уровень и т. д.). Ограничимся тремя уровнями вложенности.

1)Заголовок (может быть в несколько строк)

2)Глава/параграф/секция/подзаголовок/раздел + нумерация

3)Подглава/подсекция/подпараграф/подраздел + нумерация

Могут быть подзаголовки без нумерации и без начальных слов, просто строки текста, выделенные курсивом, подчеркнуты, жирным шрифтом + отступ (???)

#### 2)Списки (уровень вложенности?)

Будем анализировать технические отчеты и статьи законов - считаем нумерованными списками следующее:

1)Вложенные списки с нумерацией 1.1, 1.1.1, и т.д.  - это списки 2, 3 уровня. Выделим такие списки как отдельный класс с неограниченным уровнем вложенности. Уровень таких списков определяется отдельно.

2)Списки + уровень вложенности (ограничимся третьим уровнем вложенности)

-Строки, начинающиеся с цифр (нумерованные списки) 1. 1) 1

-Списки, в которых нумерация ведется с помощью букв а), А), а, А, (a), (A)

-Списки маркированные точка, галка, кружок, тире, ромб, квадрат, крестик и т.д.

При этом можно учитывать отступ от начала строки и смотреть, продолжается ли нумерация далее
Если один список вложен в другой, то его уровень увеличивается

Заголовки и элементы списков(?) могут занимать несколько строк, это нужно также уметь как-то определять

#### 3)Просто строки

### JSON на выходе

[ {"type": '', "level": '', "content": ''}, { }, { } ]

type - тип строчки - header - 0, list - 1, text - 2

level - для заголовков, списков - 1, 2, 3, ...  уровни

content - содержимое строчки

### Оценка 

f-мера макро

$F_{measure} = \frac{2*Precision*Recall}{Precision + Recall}$

$Precision = \frac{TP}{TP + FP}$

$Recall = \frac{TP}{TP + FN}$

Что делать с английскими буквами? (или римскими цифрами, например)

Как извлекать информацию о шрифтах, отступах и т.д.?

Насколько сложная структура может быть (насколько сложные и разнообразные документы)? 

Как оценивать вместе с номером уровня заголовка?


## Извлечение текста из pdf
https://www.severcart.ru/blog/all/tesseract_ocr_python/

In [1]:
from pdf2text import pdf2text

## Правила для извлечения логической структуры

In [27]:
import re
RE_LIST = re.compile(r'\d+(\.\d+)*\D') # для отдельного типа списка

In [42]:
match = RE_LIST.match('1.1.1 kljkhg')
match.string[match.span()[0]:match.span()[1]].count('.')

2

In [25]:
def list_proc(line):
    """
    если список, возвращает тип и глубину вложенности
    """
    match = RE_LIST.search(line)
    if match:
        if match.start() == 0:
            return 'list', match.string[match.span()[0]:match.span()[1]].count('.') + 1
    return

pipeline - классификатор, на входе - список строк текста, на выходе - тип строк

In [57]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
class string2features:
    def __init__(self):
        pass
    def fit(self):
        pass
    def predict(self):
        pass
    def fit_transform(self, X, y):
        return self.transform(X)
    def transform(self, X):
        """
        X - список строк
        """
        first_words = []
        for line in X:
            if line.split():
                first_words.append(line.split()[0])
            else:
                first_words.append('')
        return first_words

ppl = make_pipeline(string2features(), CountVectorizer(token_pattern=r'(?u)\b\w+\b'), LogisticRegression())

Извлекаем признаки:
смотрим на первое слово в каждой непустой строчке документа -> считаем слова

In [3]:
import cv2
import pytesseract
image = cv2.imread('0004.jpeg')
text = pytesseract.image_to_string(image, lang='rus') #to_data
lines = text.split(sep='\n')
first_words = []
for line in lines:
    if line.split():
        first_words.append(line.split()[0])
    else:
        first_words.append('')
first_words

['1.',
 '',
 '1.1.',
 'Доработка',
 '',
 'системы',
 'зав.',
 '',
 '1.2.',
 '',
 '—',
 '13ДГОЗ',
 '',
 '—',
 'ное',
 '',
 '1.3.',
 '',
 '1.4.',
 '',
 '2.',
 '',
 '2.1.',
 'документации',
 'пункта',
 '№802),',
 'корабельными',
 'ной',
 'верное',
 'екта,',
 'кументации.',
 '',
 '2.2.',
 'командного',
 '',
 '2.3.']

### Разметка

вектор ответов 

1-header -> level

2-list -> level 

3-text


In [7]:
import json
doc_with_labels = []
for line in lines:
    print(line)
    type_ = int(input())
    if type_ == 1 or type_ == 2:
        level_ = int(input())
    else:
        level_ = None
    doc_with_labels.append({"type": type_, "level": level_, "content": line})
with open("file_with_labels.json", "w") as write_file:
    json.dump(doc_with_labels, write_file, ensure_ascii=False)
doc_with_labels

1. Наименование работы и основание для ее выполнения
2
1

3
1.1. Наименование работы
2
2
Доработка конструкторской документации интегрированной мостиковой
3

3
системы ходового командного пункта (ИМС ХКП) для заказа 11442М
3
зав. №802.
3

3
1.2. Основание для выполнения работы:
2
2

3
— Государственный контракт на выполнение работ №Р/1/1/0122/ГК-
2
3
13ДГОЗ 2013 г.
3

3
— Тактико-техническое задание модернизации заказа 11442, утвержден-
2
3
ное Министром обороны РФ 6.05.2013 года.
3

3
1.3. Исполнитель —
2
2

3
1.4. Заказчик — ОАО «Северное ПКБ», г. Санкт-Петербург.
2
2

3
2. Цель выполнения работы, наименование и индекс системы
2
1

3
2.1. Целью работы является доработка Исполнителем конструкторской
2
2
документации интегрированной мостиковой системы ходового командного
3
пункта по техническим условиям ИСПОЛНИТЕЛЯ ддля заказа 11442М (зав.
3
№802), решение вопросов электрического и конструктивного сопряжения с
3
корабельными системами, определение состава (комплектации) доработан-
3
но

[{'type': 2,
  'level': 1,
  'content': '1. Наименование работы и основание для ее выполнения'},
 {'type': 3, 'level': None, 'content': ''},
 {'type': 2, 'level': 2, 'content': '1.1. Наименование работы'},
 {'type': 3,
  'level': None,
  'content': 'Доработка конструкторской документации интегрированной мостиковой'},
 {'type': 3, 'level': None, 'content': ''},
 {'type': 3,
  'level': None,
  'content': 'системы ходового командного пункта (ИМС ХКП) для заказа 11442М'},
 {'type': 3, 'level': None, 'content': 'зав. №802.'},
 {'type': 3, 'level': None, 'content': ''},
 {'type': 2, 'level': 2, 'content': '1.2. Основание для выполнения работы:'},
 {'type': 3, 'level': None, 'content': ''},
 {'type': 2,
  'level': 3,
  'content': '— Государственный контракт на выполнение работ №Р/1/1/0122/ГК-'},
 {'type': 3, 'level': None, 'content': '13ДГОЗ 2013 г.'},
 {'type': 3, 'level': None, 'content': ''},
 {'type': 2,
  'level': 3,
  'content': '— Тактико-техническое задание модернизации заказа 11442, 

Обучаем модель:
логистическая регрессия, для каждой строки - тип строки + уровень???

как работать с уровнем? строка-название + строка-уровень

In [54]:
import json
with open("file_with_labels.json", "r") as read_file:
    doc_with_labels = json.load(read_file)

In [55]:
y = [x["type"] for x in doc_with_labels]
X = [x["content"] for x in doc_with_labels]
X, y

(['1. Наименование работы и основание для ее выполнения',
  '',
  '1.1. Наименование работы',
  'Доработка конструкторской документации интегрированной мостиковой',
  '',
  'системы ходового командного пункта (ИМС ХКП) для заказа 11442М',
  'зав. №802.',
  '',
  '1.2. Основание для выполнения работы:',
  '',
  '— Государственный контракт на выполнение работ №Р/1/1/0122/ГК-',
  '13ДГОЗ 2013 г.',
  '',
  '— Тактико-техническое задание модернизации заказа 11442, утвержден-',
  'ное Министром обороны РФ 6.05.2013 года.',
  '',
  '1.3. Исполнитель —',
  '',
  '1.4. Заказчик — ОАО «Северное ПКБ», г. Санкт-Петербург.',
  '',
  '2. Цель выполнения работы, наименование и индекс системы',
  '',
  '2.1. Целью работы является доработка Исполнителем конструкторской',
  'документации интегрированной мостиковой системы ходового командного',
  'пункта по техническим условиям ИСПОЛНИТЕЛЯ ддля заказа 11442М (зав.',
  '№802), решение вопросов электрического и конструктивного сопряжения с',
  'корабельным

Сохраняем и загружаем обученную модель

In [58]:
import pickle as pkl
clf = ppl.fit(X, y)
pkl.dump(clf, open("model.pkl", "wb"))
model = pkl.load(open("model.pkl", "rb"))

/home/nastyboget/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


предскажем классы по тем же признакам (разделить трэйн и тест) ??? 

In [60]:
y_pred = clf.predict(X)
y_pred

array([2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2])

## Оценка

пока качество определения уровня заголовка не учитываю и не строю дерево

In [61]:
from sklearn.metrics import f1_score
f1_score(y, y_pred, average='macro')

0.9307692307692308